# Analyze Vdbench Logs

In [ ]:
# S3_LOGS_URLS = ['412URL', '413URL'] - for comparing 2 artifacts 
S3_LOGS_URLS = ['']
# Compare versions
versions = ['412', '413']

# Prerequsite

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

# extract credentials
import configparser
config = configparser.ConfigParser()
config.read('logs.conf')
username = config.get("data", 'user') 
password = config.get("data", 'password')


# Update fedora
!dnf update -y


from benchmark_runner.jupyterlab.templates.logs_operations.logs_operations import LogsOperations
from benchmark_runner.jupyterlab.templates.analyze_workloads.analyze_vdbench_logs.analyze_vdbench_logs import AnalyzeVdbenchLogs

# Logs operations
logs_operations_obj = []
analyze_vdbench_logs_obj = []
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations = LogsOperations(s3_logs_url=S3_LOGS_URL)
    logs_operations_obj.append(logs_operations)

# Logs cleanup
logs_operations_obj[0].cleanup()

for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs = AnalyzeVdbenchLogs(s3_logs_url=S3_LOGS_URL)
    analyze_vdbench_logs_obj.append(analyze_vdbench_logs)

# Output clean output display
analyze_vdbench_logs_obj[0].clear_output()

## Download S3 logs

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].download_s3_logs(username=username, password=password)

## Untar logs & chmod 

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    logs_operations_obj[i].untar_and_chmod_logs()

# Get vdbench log files

In [ ]:
csv_log_files=[]
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    csv_log_files.append(analyze_vdbench_logs_obj[i].get_vdbench_csv_log_files())

# Display Dataframe

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_df()

## Vdbench latency (sec)

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_latency_horizontal()

## Vdbench IOPS

In [ ]:
for i, S3_LOGS_URL in enumerate(S3_LOGS_URLS):
    analyze_vdbench_logs_obj[i].display_iops_horizontal()

# Compare Vdbench Latency (sec) - Lower is better

In [ ]:
if len(S3_LOGS_URLS)>1:
    analyze_vdbench_logs_obj[0].compare_latency(csv_log_files1=csv_log_files[0], csv_log_files2=csv_log_files[1], legend_label1=versions[0], legend_label2=versions[1])

# Compare Vdbench IOPS - Hight is better

In [ ]:
if len(S3_LOGS_URLS)>1:
    analyze_vdbench_logs_obj[0].compare_iops(csv_log_files1=csv_log_files[0], csv_log_files2=csv_log_files[1], legend_label1=versions[0], legend_label2=versions[1])